### This notebook demonstrates how to build a multi-agent environment.
- It explains the concepts of manager orchestration (centralized control) and decentralized orchestration (agents delegate among themselves).
- We implemented frontend and backend engineer agents that hand off tasks to each other, illustrating agentic handoff.
- The notebook also covers a decentralized email writing workflow: a delegation agent assigns email writing to one of three agents (professional, busy, or humorous writer), who then hand off the completed email to an email sending agent.

In [ ]:
from agents import Agent, Runner, function_tool

In [ ]:
# Build frontend and backend engineer agents. Backend engineer agent is responsible for building the backend of the application. If the task is not related to backend, delegate it to an appropriate agent.

frontend_engineer_agent = Agent(
    name="Frontend Engineer",
    instructions="You are a frontend engineer. You are responsible for building the frontend of the application. If the task is not related to frontend, delegate it to an appropriate agent.",
    model="gpt-4o-mini",
    handoff_description="You are not a frontend engineer and you want to delegate the task to an appropriate frontend engineer agent.",
)

backend_engineer_agent = Agent(
    name="Backend Engineer",
    instructions="You are a backend engineer. You are responsible for building the backend of the application. If the task is not related to backend, delegate it to an appropriate agent.",
    model="gpt-4o-mini",
    handoff_description="You are not a backend engineer and you want to delegate the task to an appropriate backend engineer agent.",
    handoffs=[frontend_engineer_agent]
)

In [ ]:
# Run the backend engineer agent to build the backend of the application or delegate the task to an appropriate agent.

result = await Runner.run(starting_agent=backend_engineer_agent, input="Write code snippet for encrypting user's password using bcrypt on backend.")
print(result.final_output)

result = await Runner.run(starting_agent=backend_engineer_agent, input="Write a component in React JS for a login page.")
print(result.final_output)

In [ ]:
# Let's build something more interesting. We will build a decentralized email writing workflow.
# - Write a tool to send an email.
# - Build an email sending agent that sends the final email to the customer.
# - Build three different agents: professional, busy, and humorous writer. They write emails based on the context provided by the user and handoff the email content to the email sending agent to send the email to the customer.
# - Build a delegation agent that assigns the email writing task to one of the three agents based on the context provided by the user.
# Observe the agent-loop and handoffs through OpenAI traces.

@function_tool
def send_email(email_address: str, subject: str, body: str):
    """
    Send an email to the customer
    Args:
        email_address: str
        subject: str
        body: str
    Returns:
        str: "Email sent successfully"
    """
    return "Email sent successfully"

email_sending_agent = Agent(
    name="Email Sending Agent",
    instructions="You are a email sending agent. You are responsible for sending the email to the customer based on the context provided by other agents.",
    model="gpt-4o-mini",
    handoff_description="Delegate to this agent for sending the final email to the customer",
    tools=[send_email]
)

professional_sdr_agent = Agent(
    name="Professional SDR",
    instructions="You are a professional SDR. You are responsible for writing professional emails to the customers based on the context provided by the user. You then handoff the email content to the Email Sending Agent to send the email to the customer. You do not send the email yourself.",
    model="gpt-4o-mini",
    handoff_description="Agent for writing a professional email to the customer",
    handoffs=[email_sending_agent]
)

busy_sdr_agent = Agent(
    name="Busy SDR",
    instructions="You are a busy SDR. You are responsible for writing concise and short emails to the customers based on the context provided by the user.  You then handoff the email content to the Email Sending Agent to send the email to the customer. You do not send the email yourself.",
    model="gpt-4o-mini",
    handoff_description="Agent for writing a short and concise email to the customer",
    handoffs=[email_sending_agent]
)

humorous_sdr_agent = Agent(
    name="Humorous SDR",
    instructions="You are a humorous SDR. You are responsible for writing humorous and engaging emails to the customers based on the context provided by the user. You then handoff the email content to the Email Sending Agent to send the email to the customer. You do not send the email yourself.",
    model="gpt-4o-mini",
    handoff_description="Agent for writing a humorous email to the customer",
    handoffs=[email_sending_agent]
)

email_delegation_agent = Agent(
    name="Email Delegation Agent",
    instructions="You are a email delegation agent. You are responsible for delegating the email writing task to the appropriate agent based on the context provided by the user.",
    model="gpt-4o-mini",
    handoffs=[professional_sdr_agent, busy_sdr_agent, humorous_sdr_agent]
)

In [ ]:
# Run the delegation agent to assign the email writing task to one of the three agents based on the context provided by the user.

result = await Runner.run(starting_agent=email_delegation_agent, input="Write a cold email to Moonis. He is a very non-serious and memer person. Tell him about our new grinder and blender products line. These products belong to ByteMage. His email is moonis@bytemage.io.")
print(result.final_output)